<h1> Implementing matrix factorization by using QR decomposition </H1>

In [43]:
# Clone the entire repo.
!git clone -l -s https://github.com/Chinmay-jain767/Recommendation_Engine.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 20 (delta 2), reused 15 (delta 1), pack-reused 0
Unpacking objects: 100% (20/20), done.
/resources/cloned-repo/cloned-repo
LICENSE  MovieLen100k  MovieLen1M  README.md


In [44]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm

In [45]:
new_header = ['MovieID','Title','Genres'] 
new_header2=['UserID','Gender','Age','Occupation','Zip-code']
new_header3=['UserID','MovieID','Rating','Timestamp']


In [46]:
Movies = pd.read_csv("./MovieLen1M/movies.dat", sep='::',header=None,names=new_header)
Ratings = pd.read_csv("./MovieLen1M/ratings.dat", sep='::',header=None,names=new_header3)
Links = pd.read_csv("./MovieLen1M/users.dat", sep='::',header=None,names=new_header2)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warni

In [47]:
Movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [48]:
Links.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [49]:
Ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


<h3> combining movies and ratings data into one dataframe </h3>

In [50]:
df1=pd.merge(Movies, Ratings, on='MovieID', how='inner')

In [51]:
df1.head()

,MovieID,Title,Genres,UserID,Rating,Timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268
1,1,Toy Story (1995),Animation|Children's|Comedy,6,4,978237008
2,1,Toy Story (1995),Animation|Children's|Comedy,8,4,978233496
3,1,Toy Story (1995),Animation|Children's|Comedy,9,5,978225952
4,1,Toy Story (1995),Animation|Children's|Comedy,10,5,978226474


<h4> removing movieId and timestamp from df1 </h4>

In [52]:
df2=df1.drop(['MovieID','Timestamp'],axis=1)

In [53]:
df2.head()

,Title,Genres,UserID,Rating
0,Toy Story (1995),Animation|Children's|Comedy,1,5
1,Toy Story (1995),Animation|Children's|Comedy,6,4
2,Toy Story (1995),Animation|Children's|Comedy,8,4
3,Toy Story (1995),Animation|Children's|Comedy,9,5
4,Toy Story (1995),Animation|Children's|Comedy,10,5


<h3> dataframe of users giving rating to each movie</h3>

In [54]:
user_rating=df2.pivot(index='UserID',columns='Title',values='Rating').fillna(0)

In [55]:
user_rating

Title,"$1,000,000 Duck (1971)",'Night Mother (1986),'Til There Was You (1997),"'burbs, The (1989)",...And Justice for All (1979),1-900 (1994),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),...,"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),Young and Innocent (1937),Your Friends and Neighbors (1998),Zachariah (1971),"Zed & Two Noughts, A (1985)",Zero Effect (1998),Zero Kelvin (Kj�rlighetens kj�tere) (1995),Zeus and Roxanne (1997),eXistenZ (1999)
UserID,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.0,3.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
df3 = user_rating.as_matrix()
user_ratings_mean = np.mean(df3, axis = 1)
R_demeaned = df3 - user_ratings_mean.reshape(-1, 1)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [57]:
df3

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [58]:
R_demeaned

array([[-0.05990286, -0.05990286, -0.05990286, ..., -0.05990286,
        -0.05990286, -0.05990286],
       [-0.12924987, -0.12924987, -0.12924987, ..., -0.12924987,
        -0.12924987, -0.12924987],
       [-0.05369671, -0.05369671, -0.05369671, ..., -0.05369671,
        -0.05369671, -0.05369671],
       ...,
       [-0.02050729, -0.02050729, -0.02050729, ..., -0.02050729,
        -0.02050729, -0.02050729],
       [-0.1287102 , -0.1287102 , -0.1287102 , ..., -0.1287102 ,
        -0.1287102 , -0.1287102 ],
       [-0.3291959 , -0.3291959 , -0.3291959 , ..., -0.3291959 ,
        -0.3291959 , -0.3291959 ]])

<h2>Singular Value Decomposition</h2>

In [16]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 50)

In [17]:
sigma = np.diag(sigma)


In [18]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = user_rating.columns)

In [19]:
preds_df

Title,"$1,000,000 Duck (1971)",'Night Mother (1986),'Til There Was You (1997),"'burbs, The (1989)",...And Justice for All (1979),1-900 (1994),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),...,"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),Young and Innocent (1937),Your Friends and Neighbors (1998),Zachariah (1971),"Zed & Two Noughts, A (1985)",Zero Effect (1998),Zero Kelvin (Kj�rlighetens kj�tere) (1995),Zeus and Roxanne (1997),eXistenZ (1999)
0,0.007025,0.092980,-0.000116,0.039330,0.164734,-0.003143,0.211770,1.548461,0.474657,0.670314,...,0.067065,-0.135281,-0.010072,0.010096,-0.010447,0.028755,0.143893,0.024624,0.022066,0.034644
1,0.008243,0.186769,-0.043163,0.022736,0.175550,0.001458,-0.148779,-0.022620,0.030157,1.215488,...,-0.103296,-0.080331,0.006779,-0.170137,0.009426,-0.041060,-0.220015,-0.005832,-0.016048,-0.203493
2,-0.003647,-0.026550,-0.047464,0.034431,0.079607,-0.003605,-0.107924,0.064833,0.234843,-0.196047,...,0.056556,0.091845,-0.015356,-0.101750,0.007945,0.003175,-0.069349,0.007852,-0.037541,-0.173971
3,-0.016454,-0.005114,-0.014119,-0.090782,-0.081278,-0.012863,-0.032037,0.142567,-0.035012,-0.030336,...,0.019859,-0.079456,-0.026522,0.044697,-0.011860,0.000450,-0.019497,-0.011953,0.003523,0.070770
4,0.080431,0.088529,0.010178,0.024978,-0.287260,0.029600,0.498140,-0.054863,0.163152,0.151199,...,0.408928,0.092444,0.003205,0.445252,0.038909,0.082954,0.849553,0.007851,0.046094,0.864229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.168056,0.705550,0.066947,0.291193,0.869680,-0.024583,1.738268,2.956460,0.164889,1.937207,...,0.660366,1.181018,0.127948,0.773255,-0.006621,0.441392,2.022599,0.065749,0.090097,2.678104
6036,0.112755,0.109249,0.010983,0.255103,0.521570,-0.011751,0.196600,-0.030842,0.175396,2.930876,...,-0.075262,0.156687,0.089793,-0.105415,-0.022303,-0.021764,0.357722,-0.039262,0.061136,0.384860
6037,0.005807,-0.005630,0.001908,0.038212,-0.034169,0.003409,0.000193,0.081951,-0.090715,-0.116725,...,0.033639,-0.054348,0.004636,-0.006644,0.001553,0.017555,-0.029321,0.002574,0.000389,-0.026132
6038,0.021020,-0.016563,0.032413,0.054021,0.026687,-0.005317,0.094341,0.792178,0.035070,0.643813,...,0.016624,0.187358,0.044957,0.066898,-0.004111,-0.025725,0.060461,-0.004757,-0.019230,-0.154440


<h1> QR decomposition </h1>

In [59]:
q,r=np.linalg.qr(R_demeaned)

In [60]:
r

array([[ 2.47826014e+01,  8.11930654e+00,  9.63849683e+00, ...,
         1.13526910e+01,  1.16996477e+01, -6.07688967e+00],
       [ 0.00000000e+00,  3.00388849e+01,  4.78211227e+00, ...,
         3.86876688e+00,  3.02923643e+00, -1.63856946e+00],
       [ 0.00000000e+00,  0.00000000e+00,  2.30171132e+01, ...,
         6.30466356e+00,  5.92534045e+00, -1.62937624e+00],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         2.54077797e-01, -1.20149639e-01, -1.33928158e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00, -3.98089028e+00,  3.98089028e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00, -3.36464067e-12]])

In [61]:
q

array([[-2.41713367e-03, -1.34084242e-03, -1.31177467e-03, ...,
         4.38011758e-03, -1.84662909e-02, -1.63426514e-01],
       [-5.21534697e-03, -2.89307891e-03, -2.83036067e-03, ...,
        -1.28169891e-02,  7.61141529e-03,  7.36504517e-02],
       [-2.16670991e-03, -1.20192631e-03, -1.17587009e-03, ...,
        -3.05325438e-03, -3.77662011e-03,  1.72217596e-01],
       ...,
       [-8.27487201e-04, -4.59027134e-04, -4.49076014e-04, ...,
         4.66065601e-03, -6.84572551e-03, -1.63618022e-02],
       [-5.19357099e-03, -2.88099925e-03, -2.81854288e-03, ...,
        -1.84993548e-02,  7.58116018e-03, -1.21087323e-02],
       [-1.32833472e-02, -7.36859347e-03, -7.20885180e-03, ...,
         1.33814727e-02, -7.03513388e-03,  8.11786765e-05]])

In [62]:
prediction=np.dot(q,r)

In [66]:
prediction

array([[-0.05990286, -0.05990286, -0.05990286, ..., -0.05990286,
        -0.05990286, -0.05990286],
       [-0.12924987, -0.12924987, -0.12924987, ..., -0.12924987,
        -0.12924987, -0.12924987],
       [-0.05369671, -0.05369671, -0.05369671, ..., -0.05369671,
        -0.05369671, -0.05369671],
       ...,
       [-0.02050729, -0.02050729, -0.02050729, ..., -0.02050729,
        -0.02050729, -0.02050729],
       [-0.1287102 , -0.1287102 , -0.1287102 , ..., -0.1287102 ,
        -0.1287102 , -0.1287102 ],
       [-0.3291959 , -0.3291959 , -0.3291959 , ..., -0.3291959 ,
        -0.3291959 , -0.3291959 ]])

In [67]:
prediction_df=pd.DataFrame(prediction, columns = user_rating.columns)

In [68]:
prediction_df

Title,"$1,000,000 Duck (1971)",'Night Mother (1986),'Til There Was You (1997),"'burbs, The (1989)",...And Justice for All (1979),1-900 (1994),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),...,"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),Young and Innocent (1937),Your Friends and Neighbors (1998),Zachariah (1971),"Zed & Two Noughts, A (1985)",Zero Effect (1998),Zero Kelvin (Kj�rlighetens kj�tere) (1995),Zeus and Roxanne (1997),eXistenZ (1999)
0,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,...,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903
1,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,...,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250
2,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,...,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697
3,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,...,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745
4,-0.168106,-0.168106,-0.168106,-0.168106,-0.168106,-0.168106,-0.168106,-0.168106,-0.168106,-0.168106,...,-0.168106,-0.168106,-0.168106,-0.168106,-0.168106,-0.168106,-0.168106,-0.168106,-0.168106,-0.168106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,-0.791419,2.208581,-0.791419,-0.791419,-0.791419,-0.791419,1.208581,3.208581,-0.791419,-0.791419,...,-0.791419,2.208581,-0.791419,-0.791419,-0.791419,-0.791419,-0.791419,-0.791419,-0.791419,1.208581
6036,-0.202644,-0.202644,-0.202644,-0.202644,-0.202644,-0.202644,-0.202644,-0.202644,-0.202644,3.797356,...,-0.202644,-0.202644,-0.202644,-0.202644,-0.202644,-0.202644,-0.202644,-0.202644,-0.202644,-0.202644
6037,-0.020507,-0.020507,-0.020507,-0.020507,-0.020507,-0.020507,-0.020507,-0.020507,-0.020507,-0.020507,...,-0.020507,-0.020507,-0.020507,-0.020507,-0.020507,-0.020507,-0.020507,-0.020507,-0.020507,-0.020507
6038,-0.128710,-0.128710,-0.128710,-0.128710,-0.128710,-0.128710,-0.128710,-0.128710,-0.128710,-0.128710,...,-0.128710,2.871290,-0.128710,-0.128710,-0.128710,-0.128710,-0.128710,-0.128710,-0.128710,-0.128710


<h1>Making Movie Recommendations</h1>

In [129]:
def recommend_movies(predictions_df, userID, movies_df, original_ratings_df, num_recommendations):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # UserID starts at 1, not 0
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.UserID == (userID)]
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'MovieID', right_on = 'MovieID').
                     sort_values(['Rating'], ascending=False)
                 )

    print ('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print ('Recommending the highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies_df[~movies_df['MovieID'].isin(user_full['MovieID'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'Title',right_on = 'Title' ).
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )



    return user_full,recommendations

already_rated_QR, predictions_QR = recommend_movies(prediction_df, 837, Movies, Ratings, 5)

User 837 has already rated 69 movies.
Recommending the highest 5 predicted ratings movies not already rated.


In [135]:
already_rated_QR.head(10)

,UserID,MovieID,Rating,Timestamp,Title,Genres
36,837,858,5,975360036,"Godfather, The (1972)",Action|Crime|Drama
35,837,1387,5,975360036,Jaws (1975),Action|Horror
65,837,2028,5,975360089,Saving Private Ryan (1998),Action|Drama|War
63,837,1221,5,975360036,"Godfather: Part II, The (1974)",Action|Crime|Drama
11,837,913,5,975359921,"Maltese Falcon, The (1941)",Film-Noir|Mystery
20,837,3417,5,975360893,"Crimson Pirate, The (1952)",Adventure|Comedy|Sci-Fi
34,837,2186,4,975359955,Strangers on a Train (1951),Film-Noir|Thriller
55,837,2791,4,975360893,Airplane! (1980),Comedy
31,837,1188,4,975360920,Strictly Ballroom (1992),Comedy|Romance
28,837,1304,4,975360058,Butch Cassidy and the Sundance Kid (1969),Action|Comedy|Western


In [136]:
predictions_QR

,MovieID,Title,Genres
1192,1235,Harold and Maude (1971),Comedy
1064,1096,Sophie's Choice (1982),Drama
3671,3809,What About Bob? (1991),Comedy
1601,1682,"Truman Show, The (1998)",Drama
1340,1393,Jerry Maguire (1996),Drama|Romance


In [132]:

    already_rated_SVD, predictions_SVD = recommend_movies(preds_df, 310, Movies, Ratings, 5)
  


User 310 has already rated 223 movies.
Recommending the highest 5 predicted ratings movies not already rated.


In [133]:
already_rated_SVD.head()

,UserID,MovieID,Rating,Timestamp,Title,Genres
0,310,719,5,976478304,Multiplicity (1996),Comedy
43,310,2236,5,976477531,Simon Birch (1998),Drama
133,310,3461,5,976481230,Lord of the Flies (1963),Adventure|Drama|Thriller
124,310,1777,5,976477459,"Wedding Singer, The (1998)",Comedy|Romance
122,310,2572,5,976477596,10 Things I Hate About You (1999),Comedy|Romance


In [134]:
predictions_SVD

,MovieID,Title,Genres
294,318,"Shawshank Redemption, The (1994)",Drama
313,339,While You Were Sleeping (1995),Comedy|Romance
27,34,Babe (1995),Children's|Comedy|Drama
1820,2028,Saving Private Ryan (1998),Action|Drama|War
1102,1198,Raiders of the Lost Ark (1981),Action|Adventure


<h1> References: </h1>

https://codeburst.io/explaination-of-recommender-systems-in-information-retrieval-13077e1d916c


https://www.youtube.com/watch?v=ZspR5PZemcs&t=1330s